<a href="https://colab.research.google.com/github/tienhuynh96/NLP_Projects/blob/main/Demo_Medical_NER_Maccrobat2018_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1. Data preparation

##1.1 Load data

In [2]:
# Download dataset and copy dataset into Colab
# https://figshare.com/articles/dataset/MACCROBAT2018/9764942
# https://brat.nlplab.org/standoff.html

In [3]:
# Make directory MACCROBAT2018
!mkdir MACCROBAT2018

In [15]:
# Unzip MACCROBAT2018.zip and place the extracted files and directories into the ./MACCROBAT2018 directory
!unzip /content/MACCROBAT2018.zip -d ./MACCROBAT2018

Archive:  /content/MACCROBAT2018.zip
  inflating: ./MACCROBAT2018/15939911.ann  
  inflating: ./MACCROBAT2018/15939911.txt  
  inflating: ./MACCROBAT2018/16778410.ann  
  inflating: ./MACCROBAT2018/16778410.txt  
  inflating: ./MACCROBAT2018/17803823.ann  
  inflating: ./MACCROBAT2018/17803823.txt  
  inflating: ./MACCROBAT2018/18236639.ann  
  inflating: ./MACCROBAT2018/18236639.txt  
  inflating: ./MACCROBAT2018/18258107.ann  
  inflating: ./MACCROBAT2018/18258107.txt  
  inflating: ./MACCROBAT2018/18416479.ann  
  inflating: ./MACCROBAT2018/18416479.txt  
  inflating: ./MACCROBAT2018/18561524.ann  
  inflating: ./MACCROBAT2018/18561524.txt  
  inflating: ./MACCROBAT2018/18666334.ann  
  inflating: ./MACCROBAT2018/18666334.txt  
  inflating: ./MACCROBAT2018/18787726.ann  
  inflating: ./MACCROBAT2018/18787726.txt  
  inflating: ./MACCROBAT2018/18815636.ann  
  inflating: ./MACCROBAT2018/18815636.txt  
  inflating: ./MACCROBAT2018/19009665.ann  
  inflating: ./MACCROBAT2018/19009665.t

In [5]:
!pip -q install evaluate accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 33.1 MB/s eta 0:00:00


In [11]:
ls

MACCROBAT2018/  MACCROBAT2018.zip  sample_data/


In [7]:
# Check
import os
from typing import List, Dict, Tuple

# Create dataset_folder
dataset_folder = "./MACCROBAT2018"
# Get file name into file_ids list, Eg: [15939911,...]
file_ids = [f.split(".")[0] for f in os.listdir(dataset_folder) if f.endswith('.txt')]

# Create .txt file name for input
text_files = [f+".txt" for f in file_ids]
# Create .ann file name for label
anno_files = [f+".ann" for f in file_ids]

# Get number of samples
num_samples = len(file_ids)
# Create an empty texts (list[str])
texts: List[str] = []
# Append all .txt file into texts
for i in range(num_samples):
    file_path = os.path.join(dataset_folder, text_files[i])
    with open(file_path, 'r') as f:
      texts.append(f.read())

print(texts[0])
print(len(texts))

IndexError: list index out of range

In [ ]:
# Check
# Read all the label file and append into tags (a list of str)
tags: List[Dict[str, str]] = []
for i in range(num_samples):
    file_path = os.path.join(dataset_folder, anno_files[i])
    with open(file_path, 'r') as f:
        # \t is tab and \n is newline
        # Creating text_bound_ann contains the line with start with "T" and split them by tab word
        text_bound_ann = [t.split("\t") for t in f.read().split("\n") if t.startswith("T")]
        # Creating text bound list (each elements in each line)
        text_bound_lst = []
        # Iterator "for" to get each line in text bound ann
        for text_b in text_bound_ann:
            # Get "annotation name" and their location in text, then split them, Eg: Organization 0 4
            label = text_b[1].split(" ")
            try:
                # Check locations of annotation in text is number
                _ = int(label[1])
                _ = int(label[2])
                # Creating a dictionary for each annotation includes: text, label, start and end position in paragraph
                tag = {
                    "text": text_b[-1],
                    "label": label[0],
                    "start": label[1],
                    "end": label[2]
                }
                # Append dic of annotation to text_bound_lst
                text_bound_lst.append(tag) # For example: 28_year-old, Age, 8, 19
            except:
                pass

        # Append text_bound_lst to tags
        tags.append(text_bound_lst)

In [ ]:
tags[0]

In [ ]:
# Check
# Iterator for get full text and tags in each file.

# Get each text samples
full_text = texts[0]
# Get each tag samples
tags = tags[0]

# Create a label offset for example: 4,5,6
label_offset = []
# Create a continous label offset example: 4,5,6
continuous_label_offset = []
# Interator to get the offset where contains objects
for tag in tags:
    offset = list(range(int(tag["start"]), int(tag["end"])+1))
    label_offset.append(offset)
    continuous_label_offset.extend(offset)


In [ ]:
label_offset

In [ ]:
continuous_label_offset

In [ ]:
# Get all offset of each text
all_offset = list(range(len(full_text)))
print(all_offset)
# Finding the zero offset in text
zero_offset = [offset for offset in all_offset if offset not in continuous_label_offset]
print(zero_offset)

# Fine each continous ranges in zero offset
zero_offset = Preprocessing_Maccrobat.find_continuous_ranges(zero_offset)



##1.2 Preprocessing data (prepare data)



In [12]:
import os
from typing import List, Dict, Tuple

class Preprocessing_Maccrobat:
    def __init__(self, dataset_folder, tokenizer):
        # Creating list of file name in dataset_folder with type is .txt
        # f.split(".")[0] is standing for getting file name only
        self.file_ids = [f.split(".")[0] for f in os.listdir(dataset_folder) if f.endswith('.txt')]

        # Create .txt file name for input
        self.text_files = [f+".txt" for f in self.file_ids]
        # Create .ann file name for label
        self.anno_files = [f+".ann" for f in self.file_ids]

        # Get the number of file input
        self.num_samples = len(self.file_ids)

        # Read all the text file and append into texts (a list of str)
        self.texts: List[str] = []
        for i in range(self.num_samples):
            file_path = os.path.join(dataset_folder, self.text_files[i])
            with open(file_path, 'r') as f:
                self.texts.append(f.read())

        # Read all the label file and append into tags (a list of str)
        self.tags: List[Dict[str, str]] = []
        for i in range(self.num_samples):
            file_path = os.path.join(dataset_folder, self.anno_files[i])
            with open(file_path, 'r') as f:
                # \t is tab and \n is newline
                # Creating text_bound_ann contains the line with start with "T" and split them by tab word
                text_bound_ann = [t.split("\t") for t in f.read().split("\n") if t.startswith("T")]
                # Creating text bound list (each elements in each line)
                text_bound_lst = []
                # Iterator "for" to get each line in text bound ann
                for text_b in text_bound_ann:
                    # Get "annotation name" and their location in text, then split them, Eg: Organization 0 4
                    label = text_b[1].split(" ")
                    try:
                        # Check locations of annotation in text is number
                        _ = int(label[1])
                        _ = int(label[2])
                        # Creating a dictionary for each annotation includes: text, label, start and end position in paragraph
                        tag = {
                            "text": text_b[-1],
                            "label": label[0],
                            "start": label[1],
                            "end": label[2]
                        }
                        # Append dic of annotation to text_bound_lst
                        text_bound_lst.append(tag) # For example: 28_year-old, Age, 8, 19
                    except:
                        pass

                # Append text_bound_lst to tags
                self.tags.append(text_bound_lst)
        # Setting tokenizer
        self.tokenizer = tokenizer

    # Creating process function to pre-process the tags and return a tuple of two list[list[str]]
    def process(self) -> Tuple[List[List[str]], List[List[str]]]:
        input_texts = []
        input_labels = []

        # Iterator for get full text and tags in each file.
        for idx in range(self.num_samples):
            # Get each text samples
            full_text = self.texts[idx]
            # Get each tag samples
            tags = self.tags[idx]

            # Create a label offset (location of objects) for example: 4,5,6
            label_offset = []
            # Create a continous label offset example: 4,5,6
            continuous_label_offset = []
            # Interator to get the offset where contains objects
            for tag in tags:
                # Get Offset of each objects
                offset = list(range(int(tag["start"]), int(tag["end"])+1))
                # Append each objects offset into label_offset: Eg: [1,2,3],[5,6,7],...
                label_offset.append(offset)
                # Extend objects offset into continous_label_offset: Eg: [1,2,3,5,6,7...]
                continuous_label_offset.extend(offset)

            # Get all offset of each text
            all_offset = list(range(len(full_text)))
            # Finding the zero offset in text which is not include objects
            zero_offset = [offset for offset in all_offset if offset not in continuous_label_offset]
            # Fine each continous ranges in zero offset
            zero_offset = Preprocessing_Maccrobat.find_continuous_ranges(zero_offset)

            self.tokens = []
            self.labels = []
            # Merge offset into full text
            self._merge_offset(full_text, tags, zero_offset, label_offset)
            # Check condition len of tokens is equel len of labels, if not, raise a text.... (for debugging)
            assert len(self.tokens) == len(self.labels), f"Length of tokens and labels are not equal"

            input_texts.append(self.tokens)
            input_labels.append(self.labels)

        return input_texts, input_labels

    # Merge offset into full text:
    # Tokenize, add zero and label offset
    def _merge_offset(self, full_text, tags, zero_offset, label_offset):
        # zero_offset = [[0,1,2,3],[7,8]]
        # label_offset = [[4,5,6]]

        i = j = 0
        while i < len(zero_offset) and j < len(label_offset):
            if zero_offset[i][0] < label_offset[j][0]:
                self._add_zero(full_text, zero_offset, i)
                i += 1
            else:
                self._add_label(full_text, label_offset, j, tags)
                j += 1

        while i < len(zero_offset):
            self._add_zero(full_text, zero_offset, i)
            i += 1

        while j < len(label_offset):
            self._add_label(full_text, label_offset, j, tags)
            j += 1

    # separate word and add label for zero offset
    def _add_zero(self, full_text, offset, index):
        start, *_ ,end =  offset[index] if len(offset[index]) > 1 else (offset[index][0], offset[index][0]+1)
        text = full_text[start:end]
        text_tokens = self.tokenizer.tokenize(text)

        self.tokens.extend(text_tokens)
        self.labels.extend(
            ["O"]*len(text_tokens)
        )

    # separate word and add label for label offset
    def _add_label(self, full_text, offset, index, tags):
        start, *_ ,end =  offset[index] if len(offset[index]) > 1 else (offset[index][0], offset[index][0]+1)
        text = full_text[start:end]
        text_tokens = self.tokenizer.tokenize(text)

        self.tokens.extend(text_tokens)
        self.labels.extend(
            [f"B-{tags[index]['label']}"] + [f"I-{tags[index]['label']}"]*(len(text_tokens)-1)
        )

    @staticmethod
    # Build a dictionary with input as lists of list of strings, output is a dictionary label2id
    def build_label2id(tokens: List[List[str]]):
        # Initializes an empty dictionary
        label2id = {}
        # Initializes a counter variable id_counter to keep track of unique IDs for each token.
        id_counter = 0
        # flattens the list of lists into a single list.
        for token in [token for sublist in tokens for token in sublist]:
            # If the token is not present in the dictionary, it assigns the current value of id_counter as the value for the token key in the label2id dictionary
            if token not in label2id:
                label2id[token] = id_counter
                id_counter += 1
        return label2id

    # Convert continous ranges to each range
    @staticmethod
    def find_continuous_ranges(data: List[int]):
        if not data:
            return []
        ranges = []
        start = data[0]
        prev = data[0]
        for number in data[1:]:
            if number != prev + 1:
                ranges.append(list(range(start, prev + 1)))
                start = number
            prev = number
        ranges.append(list(range(start, prev + 1)))
        return ranges



##1.2 Preprocessing data: load tokenizer, build data

In [16]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("d4data/biomedical-ner-all")

dataset_folder = "./MACCROBAT2018"

Maccrobat_builder = Preprocessing_Maccrobat(dataset_folder, tokenizer)
input_texts, input_labels = Maccrobat_builder.process()

label2id = Preprocessing_Maccrobat.build_label2id(input_labels)
id2label = {v: k for k, v in label2id.items()}

## 1.3 Split dataset into train and val set

In [17]:
from sklearn.model_selection import train_test_split


inputs_train, inputs_val, labels_train, labels_val = train_test_split(
    input_texts,
    input_labels,
    test_size=0.2,
    random_state=42
)

## 1.4 Build dataset

In [18]:
import torch
from torch.utils.data import Dataset

MAX_LEN = 512

class NER_Dataset(Dataset):
    def __init__(self, input_texts, input_labels, tokenizer, label2id, max_len=MAX_LEN):
        super().__init__()
        self.tokens = input_texts
        self.labels = input_labels
        self.tokenizer = tokenizer
        self.label2id = label2id
        self.max_len = max_len

    def __len__(self):
        return len(self.tokens)

    def __getitem__(self, idx):
        input_token = self.tokens[idx]
        label_token = [self.label2id[label] for label in self.labels[idx]]

        input_token = self.tokenizer.convert_tokens_to_ids(input_token)
        attention_mask = [1] * len(input_token)

        input_ids = self.pad_and_truncate(input_token, pad_id= self.tokenizer.pad_token_id)
        labels = self.pad_and_truncate(label_token, pad_id=0)
        attention_mask =  self.pad_and_truncate(attention_mask, pad_id=0)

        return {
            "input_ids": torch.as_tensor(input_ids),
            "labels": torch.as_tensor(labels),
            "attention_mask": torch.as_tensor(attention_mask)
            }

    def pad_and_truncate(self, inputs: List[int], pad_id: int):
        if len(inputs) < self.max_len:
            padded_inputs = inputs + [pad_id] * (self.max_len - len(inputs))
        else:
            padded_inputs = inputs[:self.max_len]
        return padded_inputs

    def label2id(self, labels: List[str]):
        return [self.label2id[label] for label in labels]

## 1.5 Dataset loader

In [19]:
train_set = NER_Dataset(inputs_train, labels_train, tokenizer, label2id)
val_set = NER_Dataset(inputs_val, labels_val, tokenizer, label2id)

## 2. Model

## 2.1 Load pre-trained model

In [20]:
from transformers import AutoModelForTokenClassification
model = AutoModelForTokenClassification.from_pretrained(
    "d4data/biomedical-ner-all",
    label2id=label2id,
    id2label=id2label,
    ignore_mismatched_sizes=True
)
model

config.json:   0%|          | 0.00/5.00k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


model.safetensors:   0%|          | 0.00/266M [00:00<?, ?B/s]

Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at d4data/biomedical-ner-all and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([84]) in the checkpoint and torch.Size([83]) in the model instantiated
- classifier.weight: found shape torch.Size([84, 768]) in the checkpoint and torch.Size([83, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


DistilBertForTokenClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
    

## 3. Training

In [21]:
import evaluate
import numpy as np

# Load accuracy
accuracy = evaluate.load("accuracy")

# Create metrics function
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    mask = labels != 0  # ignore pad_id
    predictions = np.argmax(predictions, axis=-1)
    return accuracy.compute(predictions=predictions[mask], references=labels[mask])

In [22]:
from transformers import TrainingArguments, Trainer
# Initial parameter for training
training_args = TrainingArguments(
    output_dir="out_dir",
    learning_rate=1e-4,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    # num_train_epochs=20,
    num_train_epochs=2,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    optim="adamw_torch"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_set,
    eval_dataset=val_set,
    tokenizer = tokenizer,
    compute_metrics=compute_metrics,
)

trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,1.792158,0.280257
2,No log,1.416648,0.421224


TrainOutput(global_step=20, training_loss=2.2262941360473634, metrics={'train_runtime': 1257.1913, 'train_samples_per_second': 0.255, 'train_steps_per_second': 0.016, 'total_flos': 41870224588800.0, 'train_loss': 2.2262941360473634, 'epoch': 2.0})

## 4. Inference

In [23]:
test_sentence = """A 48 year - old female presented with vaginal bleeding and abnormal Pap smears .
Upon diagnosis of invasive non - keratinizing SCC of the cervix ,
she underwent a radical hysterectomy with salpingo - oophorectomy
which demonstrated positive spread to the pelvic lymph nodes and the parametrium .
Pathological examination revealed that the tumour also extensively involved the lower uterine segment .
"""
input = torch.as_tensor([tokenizer.convert_tokens_to_ids(test_sentence.split())])

In [24]:
input = input.to("cuda")

RuntimeError: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx

In [25]:
outputs = model(input)
outputs.logits.shape

torch.Size([1, 63, 83])

In [26]:
_, preds = torch.max(outputs.logits, -1)
preds = preds[0].cpu().numpy()

In [27]:
for token, pred in zip(test_sentence.split(), preds):
    print(f"{token}\t{id2label[pred]}")

A	O
48	I-Age
year	I-Age
-	I-Age
old	I-Age
female	I-Age
presented	O
with	O
vaginal	O
bleeding	B-Sign_symptom
and	O
abnormal	B-Sign_symptom
Pap	O
smears	O
.	O
Upon	O
diagnosis	O
of	O
invasive	O
non	O
-	O
keratinizing	O
SCC	O
of	O
the	O
cervix	O
,	O
she	O
underwent	O
a	O
radical	O
hysterectomy	O
with	O
salpingo	O
-	O
oophorectomy	I-Lab_value
which	O
demonstrated	O
positive	O
spread	O
to	O
the	O
pelvic	O
lymph	O
nodes	O
and	O
the	O
parametrium	O
.	O
Pathological	O
examination	O
revealed	O
that	O
the	O
tumour	O
also	O
extensively	O
involved	O
the	O
lower	O
uterine	I-Biological_structure
segment	O
.	O


In [ ]:
for token, pred in zip(tokenizer.tokenize(test_sentence), preds):
    print(f"{token}\t{id2label[pred]}")